In [46]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import requests
from census import Census
from us import states
import os

# Census API Key
c = Census("5e99f22848e97d5bcc53f03a358b973493473cd9")

In [47]:
# Get the Data Sets from the csv
fName = ('/Rutgers/python/DoctorsofData/NCMEC_nj.csv')
# Read our Kickstarter data into pandas
njdf = pd.read_csv(fName)
njdf.head(5)

,name,age,sex,ethnicity,height,weight,eye_color,hair_color,NCMEC_case#,case_type,missing_date,missing_city,missing_county,missing_state
0,TENIKA ABREU,17,female,hispanic,70,200,brown,black,1307298,endangeredRunaway,"Sep 3, 2017 12:00:00 AM",MORRISTOWN,Morris,NJ
1,SAAD AMMAL,14,male,black,68,115,brown,brown,1307036,endangeredRunaway,"Aug 31, 2017 12:00:00 AM",PATERSON,Passaic,NJ
2,STEVEN ANDERSON,60,male,white,69,150,blue,brown,815026,endangeredMissing,"Apr 7, 1975 12:00:00 AM",NEW LISBON,Burlington,NJ
3,AMAIA ARMSTEAD WILLIAMS,16,female,black,63,100,brown,black,1305625,endangeredRunaway,"Aug 11, 2017 12:00:00 AM",ELIZABETH,Union,NJ
4,OFIR BEN-HAIM,8,female,white,24,25,brown,brown,1186183,familyAbduct,"Mar 25, 2010 12:00:00 AM",FAIR LAWN,Bergen,NJ


In [48]:
# c.sf1.get('NAME', geo={'for': 'tract:*',
#                        'in': 'state:{}'.format(states.NJ.fips)})

In [49]:
# c.sf1.state_county_tract('NAME', states.AK.fips, '170', Census.ALL)

In [50]:
# state_county(fields, state_fips, county_fips)
cData = pd.DataFrame(c.acs5.state_county(("B01003_001E","B01002_001E"), 'NJ','013' ))
print(cData)

Empty DataFrame
Columns: []
Index: []


In [51]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population", 
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]
# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,00601,17982.0,37.6,10816.0,7453.0,10861.0,60.399288
1,00602,40260.0,39.5,16079.0,8474.0,21720.0,53.949329
2,00603,52408.0,39.9,16804.0,10179.0,25459.0,48.578461
3,00606,6331.0,40.8,12512.0,5863.0,4070.0,64.286843
4,00610,28328.0,40.2,17475.0,8452.0,14005.0,49.438718


In [52]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
# census_pd.to_csv("census_data.csv", encoding="utf-8", index=False)

In [53]:
print (states.lookup('34').abbr)

NJ
